<a href="https://colab.research.google.com/github/YuanLongPeng/RosettaStone/blob/main/LSTM_Based.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Sat Apr  9 02:02:17 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8    25W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import pathlib
import random
import string
import re
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import TextVectorization

In [3]:
def loadDataset(language):
  text_file = keras.utils.get_file(
      fname=language,
      origin="https://github.com/multi30k/dataset/raw/master/data/task1/raw/"+language,
      extract=True,
      cache_dir="/content/",
  )

  import gzip

  g = gzip.GzipFile(mode="rb", fileobj=open('/content/datasets/'+language, 'rb'))
  open(r"/content/datasets/" + language + '.txt', "wb").write(g.read())

  text_file = r"/content/datasets/" + language + '.txt'

  with open(text_file) as f:
      lines = f.read().split("\n")[:-1]
  text_pairs = []
  for line in lines:
      spa = "[start] " + line + " [end]"
      text_pairs.append(spa)
  
  return text_pairs

In [4]:
train_cs = loadDataset('train.cs.gz')
train_de = loadDataset('train.de.gz')
train_en = loadDataset('train.en.gz')
train_fr = loadDataset('train.fr.gz')

val_cs = loadDataset('val.cs.gz')
val_de = loadDataset('val.de.gz')
val_en = loadDataset('val.en.gz')
val_fr = loadDataset('val.fr.gz')

32768/23001 [==========================================] - 0s 0us/step


In [5]:
len_train_cs = len(set(' '.join(train_cs).split(' '))) + 1
sequence_length_cs = max([len(pair.split(' ')) for pair in train_cs])
print(len_train_cs)
print(sequence_length_cs)

len_train_de = len(set(' '.join(train_de).split(' '))) + 1
sequence_length_de = max([len(pair.split(' ')) for pair in train_de])
print(len_train_de)
print(sequence_length_de)

len_train_en = len(set(' '.join(train_en).split(' '))) + 1
sequence_length_en = max([len(pair.split(' ')) for pair in train_en])
print(len_train_en)
print(sequence_length_en)

len_train_fr = len(set(' '.join(train_fr).split(' '))) + 1
sequence_length_fr = max([len(pair.split(' ')) for pair in train_fr])
print(len_train_fr)
print(sequence_length_fr)


30177
35
24910
41
15460
39
17007
45


In [6]:
strip_chars = string.punctuation.replace("[", "").replace("]", "")

def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(lowercase, "[%s]" % re.escape(strip_chars), "")



sequence_length = max([sequence_length_cs, sequence_length_de, sequence_length_en, sequence_length_fr])


cs_vectorization = TextVectorization(
    max_tokens=len_train_cs, output_mode="int", output_sequence_length=sequence_length_cs,
)

auto_cs_vectorization = TextVectorization(
    max_tokens=len_train_cs, output_mode="int", output_sequence_length=sequence_length_cs+1, standardize=custom_standardization,
)

de_vectorization = TextVectorization(
    max_tokens=len_train_de, output_mode="int", output_sequence_length=sequence_length_de,
)

de_de_vectorization = TextVectorization(
    max_tokens=len_train_de, output_mode="int", output_sequence_length=sequence_length_de+1, standardize=custom_standardization,
)

en_vectorization = TextVectorization(
    max_tokens=len_train_en, output_mode="int", output_sequence_length=sequence_length_en+1, standardize=custom_standardization,
)

fr_vectorization = TextVectorization(
    max_tokens=len_train_fr, output_mode="int", output_sequence_length=sequence_length_fr,
)

cs_vectorization.adapt(train_cs)
auto_cs_vectorization.adapt(train_cs)
de_vectorization.adapt(train_de)
de_de_vectorization.adapt(train_de)
en_vectorization.adapt(train_en)
fr_vectorization.adapt(train_fr)

In [7]:

batch_size = 256

def format_dataset(cs, de):
    auto_cs = auto_cs_vectorization(cs)
    input_cs = cs_vectorization(cs)
    de = de_de_vectorization(de)
    return ({"encoder_inputs": input_cs, "decoder_inputs": de[:, :-1], "autodecoder_inputs": auto_cs[:, :-1],}, {"de": de[:, 1:], "cs": auto_cs[:, 1:]})
    #return ({"encoder_inputs": input_cs, "autodecoder_inputs": auto_cs[:, :-1],}, auto_cs[:, 1:])


def make_dataset(cs_texts, de_texts):
    cs_texts = list(cs_texts)
    de_texts = list(de_texts)
    dataset = tf.data.Dataset.from_tensor_slices((cs_texts, de_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset)
    return dataset.shuffle(2048).prefetch(16).cache()


train_ds = make_dataset(train_cs, train_de)

In [8]:
for inputs, targets in train_ds.take(1):
    print(f'inputs["encoder_inputs"].shape: {inputs["encoder_inputs"].shape}')
    print(f'inputs["decoder_inputs"].shape: {inputs["decoder_inputs"].shape}')
    print(f'inputs["autodecoder_inputs"].shape: {inputs["autodecoder_inputs"].shape}')
    print(f"targets.shape: {targets['de'].shape}")
    print(f"targets.shape: {targets['cs'].shape}")

inputs["encoder_inputs"].shape: (256, 35)
inputs["decoder_inputs"].shape: (256, 41)
inputs["autodecoder_inputs"].shape: (256, 35)
targets.shape: (256, 41)
targets.shape: (256, 35)


In [9]:
HIDDEN_DIM = 1024

language_1 = keras.Input(shape=(sequence_length_cs), name="encoder_inputs")
embeddingLang_1 = layers.Embedding(len_train_cs, HIDDEN_DIM, mask_zero = True)
embeddedLang_1 = embeddingLang_1(language_1)
lstmLang_1 = layers.Bidirectional(layers.LSTM(units = int(HIDDEN_DIM / 2), return_sequences = True ))

language_enc_1 = layers.LSTM(units = HIDDEN_DIM, return_state=True)(lstmLang_1(embeddedLang_1))

encoder = keras.Model(language_1, language_enc_1)
###########################################################################################

language_2 = keras.Input(shape=(sequence_length_de), name="de_encoder_inputs")
embeddingLang_2 = layers.Embedding(len_train_de, HIDDEN_DIM, mask_zero = True)
embeddedLang_2 = embeddingLang_2(language_2)
lstmLang_2 = layers.Bidirectional(layers.LSTM(units = int(HIDDEN_DIM/2), return_sequences = True ))

language_enc_2 = layers.LSTM(units = HIDDEN_DIM, return_state=True)(layers.LSTM(units = HIDDEN_DIM, return_sequences=True)(lstmLang_2(embeddedLang_2)))

de_encoder = keras.Model(language_2, language_enc_2)

###########################################################################################
language_3 = layers.Input(shape=(sequence_length_fr), name="fr_encoder_inputs")
embeddingLang_3 = layers.Embedding(len_train_fr, HIDDEN_DIM*2, mask_zero = True)
embeddedLang_3 = embeddingLang_3(language_3)
lstmLang_3 = layers.Bidirectional(layers.LSTM(units = int(HIDDEN_DIM), return_sequences = True ))

language_enc_3 = layers.LSTM(units = HIDDEN_DIM, return_state=True)(layers.LSTM(units = HIDDEN_DIM*2, return_sequences=True)(lstmLang_3(embeddedLang_3)))

fr_encoder = keras.Model(language_3, language_enc_3)

In [10]:

decoder_inputs_1 = layers.Input(shape=(sequence_length_cs), name="autodecoder_inputs")
embeddingLang_4 = layers.Embedding(len_train_cs, HIDDEN_DIM, mask_zero = True)
embeddedLang_4 = embeddingLang_4(decoder_inputs_1)

decoder_lstm_1 = layers.LSTM(units = HIDDEN_DIM, return_sequences=True, return_state=True)
decoder_results_1 = decoder_lstm_1(embeddedLang_4, initial_state=language_enc_1[1:3])
decoder_dense_1 = layers.TimeDistributed(layer = layers.Dense(units=len_train_cs, activation='softmax'), name="cs") 
decoder_outputs_1 = decoder_dense_1(decoder_results_1[0])

###########################################################################################
decoder_inputs_2  = layers.Input(shape=(sequence_length_de), name="decoder_inputs")
embeddingLang_5 = layers.Embedding(input_dim=len_train_de, output_dim= HIDDEN_DIM, mask_zero = True)
embeddedLang_3 =  embeddingLang_5(decoder_inputs_2)

decoder_lstm_2 = layers.LSTM(units=HIDDEN_DIM, return_sequences=True, return_state=True)
decoder_results_2 = decoder_lstm_2(embeddedLang_3, initial_state=language_enc_1[1:3])
decoder_dense_2 = layers.TimeDistributed(layer = layers.Dense(units=len_train_de, activation='softmax'), name="de")
decoder_outputs_2 = decoder_dense_2(decoder_results_2[0])

###########################################################################################

model_Free = keras.Model([language_1, decoder_inputs_1, decoder_inputs_2], 
                          [decoder_outputs_1, decoder_outputs_2])

In [12]:

###########################################################################################
model_Free.compile(
  loss = ['sparse_categorical_crossentropy', 'sparse_categorical_crossentropy'],
  optimizer = "adam",
  metrics = "accuracy"
)
model_Free.summary()


model_Free.fit(train_ds, epochs=128)#, validation_data=train_ds

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_inputs (InputLayer)    [(None, 35)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 35, 1024)     30901248    ['encoder_inputs[0][0]']         
                                                                                                  
 autodecoder_inputs (InputLayer  [(None, 35)]        0           []                               
 )                                                                                                
                                                                                                  
 bidirectional (Bidirectional)  (None, 35, 1024)     6295552     ['embedding[0][0]']        

In [ ]:

###########################################################################################
from keras.backend import abs, sqrt, square, mean
import keras.backend as K

def custom_loss(y_true, y_pred):
 
    loss_square = K.square(y_pred - y_true + K.epsilon())

    loss_sqrt = K.sqrt(K.abs(y_pred - y_true) + K.epsilon())
    
    return loss_square + loss_sqrt


similarity_2 = layers.Subtract(name="zero1")([language_enc_1[1], language_enc_2[1]])
similarity_3 = layers.Subtract(name="zero2")([language_enc_1[2], language_enc_2[2]])

similarity_4 = layers.Subtract(name="zero1")([language_enc_1[1], language_enc_3[1]])
similarity_5 = layers.Subtract(name="zero2")([language_enc_1[2], language_enc_3[2]])


decoder_results_5 = decoder_lstm_1(embeddedLang_4, initial_state=language_enc_3[1:3])
decoder_outputs_5 = decoder_dense_1(decoder_results_5[0])
decoder_results_6 = decoder_lstm_2(embeddedLang_3, initial_state=language_enc_3[1:3])
decoder_outputs_6 = decoder_dense_2(decoder_results_6[0])
model_Supervise_Hmv = keras.Model([language_3, language_1, decoder_inputs_1, decoder_inputs_2], [similarity_4, similarity_5, decoder_outputs_5, decoder_outputs_6])
model_Clip_Hmv = keras.Model([language_3, language_1], [similarity_4, similarity_5])
###########################################################################################

batch_size = 128
def format_dataset(cs, de, fr, zero):
    auto_cs = auto_cs_vectorization(cs)
    input_cs = cs_vectorization(cs)
    input_fr = fr_vectorization(fr)
    de = de_de_vectorization(de)
    return ({"encoder_inputs": input_cs, "fr_encoder_inputs": input_fr, "decoder_inputs": de[:, :-1], "autodecoder_inputs": auto_cs[:, :-1],}, 
            {"de": de[:, 1:], "cs": auto_cs[:, 1:], "zero1": zero, "zero2": zero})


def make_dataset(cs_texts, de_texts, fr_texts):
    cs_texts = list(cs_texts)
    de_texts = list(de_texts)
    fr_texts = list(fr_texts)
    zero = np.zeros((len(fr_texts), HIDDEN_DIM))
    dataset = tf.data.Dataset.from_tensor_slices((cs_texts, de_texts, fr_texts, zero))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset)
    return dataset.shuffle(2048).prefetch(16).cache()


train_ds_fr = make_dataset(train_cs, train_de, train_fr)

model_Supervise_Hmv.compile(
  loss = [custom_loss, custom_loss, 'sparse_categorical_crossentropy', 'sparse_categorical_crossentropy'],
  optimizer = "adam"
)



model_Supervise_Hmv.layers[1].trainable = True

model_Supervise_Hmv.layers[3].trainable = True

model_Supervise_Hmv.layers[5].trainable = True

model_Supervise_Hmv.layers[9].trainable = True



model_Supervise_Hmv.layers[4].trainable = False

model_Supervise_Hmv.layers[8].trainable = False

model_Supervise_Hmv.layers[10].trainable = False

model_Supervise_Hmv.layers[11].trainable = False

model_Supervise_Hmv.layers[12].trainable = False

model_Supervise_Hmv.layers[13].trainable = False

model_Supervise_Hmv.layers[14].trainable = False

model_Supervise_Hmv.layers[17].trainable = False

model_Supervise_Hmv.layers[18].trainable = False

model_Supervise_Hmv.summary()

model_Supervise_Hmv.fit(train_ds_fr, epochs=0)#, validation_data=train_ds


Model: "model_8"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 fr_encoder_inputs (InputLayer)  [(None, 45)]        0           []                               
                                                                                                  
 embedding_2 (Embedding)        (None, 45, 2048)     34830336    ['fr_encoder_inputs[0][0]']      
                                                                                                  
 encoder_inputs (InputLayer)    [(None, 35)]         0           []                               
                                                                                                  
 bidirectional_2 (Bidirectional  (None, 45, 2048)    25174016    ['embedding_2[0][0]']            
 )                                                                                          

In [ ]:

model_Supervise_Hmv.fit(train_ds_fr, epochs=25)#, validation_data=train_ds

epochs = 25  # This should be at least 30 for convergence

model_Supervise_Hmv.summary()
opt = keras.optimizers.Adam(learning_rate=0.0001)
model_Supervise_Hmv.compile(loss = [custom_loss, custom_loss, 'sparse_categorical_crossentropy', 'sparse_categorical_crossentropy'], optimizer=opt)
model_Supervise_Hmv.fit(train_ds_fr, epochs=epochs)#, validation_data=train_ds

Epoch 1/25
227/227 [==============================] - 167s 575ms/step - loss: 0.0297 - zero1_loss: 0.0033 - zero2_loss: 0.0120 - cs_loss: 0.0060 - de_loss: 0.0084
Epoch 2/25
227/227 [==============================] - 131s 575ms/step - loss: 0.0430 - zero1_loss: 0.0036 - zero2_loss: 0.0148 - cs_loss: 0.0110 - de_loss: 0.0136
Epoch 3/25
227/227 [==============================] - 131s 577ms/step - loss: 0.0409 - zero1_loss: 0.0037 - zero2_loss: 0.0152 - cs_loss: 0.0099 - de_loss: 0.0122
Epoch 4/25
227/227 [==============================] - 131s 577ms/step - loss: 0.0335 - zero1_loss: 0.0035 - zero2_loss: 0.0142 - cs_loss: 0.0069 - de_loss: 0.0088
Epoch 5/25
227/227 [==============================] - 131s 577ms/step - loss: 0.0288 - zero1_loss: 0.0034 - zero2_loss: 0.0131 - cs_loss: 0.0052 - de_loss: 0.0071
Epoch 6/25
227/227 [==============================] - 131s 577ms/step - loss: 0.0258 - zero1_loss: 0.0033 - zero2_loss: 0.0120 - cs_loss: 0.0044 - de_loss: 0.0062
Epoch 7/25
227/227 [==

In [ ]:

batch_size = 256
def format_dataset(cs, de, fr, zero):
    auto_cs = auto_cs_vectorization(cs)
    input_cs = cs_vectorization(cs)
    input_fr = fr_vectorization(fr)
    de = de_de_vectorization(de)
    return ({"encoder_inputs": input_cs, "fr_encoder_inputs": input_fr,}, 
            {"zero1": zero, "zero2": zero})


def make_dataset(cs_texts, de_texts, fr_texts):
    cs_texts = list(cs_texts)
    de_texts = list(de_texts)
    fr_texts = list(fr_texts)
    zero = np.zeros((len(fr_texts), HIDDEN_DIM))
    dataset = tf.data.Dataset.from_tensor_slices((cs_texts, de_texts, fr_texts, zero))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset)
    return dataset.shuffle(2048).prefetch(16).cache()


train_ds_fr = make_dataset(train_cs, train_de, train_fr)
model_Clip_Hmv.summary()


epochs = 50

opt = keras.optimizers.Adam(learning_rate=0.0001)
model_Clip_Hmv.compile(loss = [custom_loss, custom_loss], optimizer=opt)
model_Clip_Hmv.fit(train_ds_fr, epochs=epochs)#, validation_data=train_ds

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 fr_encoder_inputs (InputLayer)  [(None, 45)]        0           []                               
                                                                                                  
 encoder_inputs (InputLayer)    [(None, 35)]         0           []                               
                                                                                                  
 embedding_2 (Embedding)        (None, 45, 2048)     34830336    ['fr_encoder_inputs[0][0]']      
                                                                                                  
 embedding (Embedding)          (None, 35, 1024)     30901248    ['encoder_inputs[0][0]']         
                                                                                            

In [ ]:

###########################################################################################
decoder_inputs_en  = layers.Input(shape=(sequence_length_en), name="decoder_inputs")
embeddingLang_en = layers.Embedding(input_dim=len_train_en, output_dim= HIDDEN_DIM, mask_zero = True)
embeddedLang_en =  embeddingLang_en(decoder_inputs_en)

decoder_lstm_en = layers.LSTM(units=HIDDEN_DIM, return_sequences=True, return_state=True)
decoder_results_en, dh1, dc1 = decoder_lstm_en(embeddedLang_en, initial_state=language_enc_1[1:3])
decoder_lstm_en_1 = layers.LSTM(units=HIDDEN_DIM, return_sequences=True, return_state=True)
decoder_results_en_1 = decoder_lstm_en_1(decoder_results_en, initial_state=language_enc_1[1:3])

decoder_dense_en = layers.Dense(units=len_train_en, activation='softmax', name="en")
decoder_outputs_en = decoder_dense_en(decoder_results_en_1[0])

###########################################################################################

model_en = keras.Model([language_1, decoder_inputs_en], 
                          [decoder_outputs_en])




batch_size = 256
def format_dataset(cs, en):
    input_cs = cs_vectorization(cs)
    en = en_vectorization(en)
    return ({"encoder_inputs": input_cs, "decoder_inputs": en[:, :-1],}, 
            {"en": en[:, 1:]})


def make_dataset(cs_texts, en_texts):
    cs_texts = list(cs_texts)
    en_texts = list(en_texts)
    dataset = tf.data.Dataset.from_tensor_slices((cs_texts, en_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset)
    return dataset.shuffle(2048).prefetch(16).cache()


train_ds_en = make_dataset(train_cs, train_en)

model_en.compile(
  loss = ['sparse_categorical_crossentropy'],
  optimizer = "adam",
  metrics = "accuracy"
)

model_en.summary()

model_en.fit(train_ds_en, epochs=0)

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_inputs (InputLayer)    [(None, 35)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 35, 1024)     30901248    ['encoder_inputs[0][0]']         
                                                                                                  
 decoder_inputs (InputLayer)    [(None, 39)]         0           []                               
                                                                                                  
 bidirectional (Bidirectional)  (None, 35, 1024)     6295552     ['embedding[0][0]']              
                                                                                            

In [ ]:
epochs = 20
opt = keras.optimizers.Adam(learning_rate=0.0001)
model_en.compile(
  loss = ['sparse_categorical_crossentropy'],
  optimizer = opt,
  metrics = "accuracy"
)
model_en.fit(train_ds_en, epochs=epochs)#, validation_data=train_ds

Epoch 1/20
114/114 [==============================] - 53s 299ms/step - loss: 0.0153 - accuracy: 0.9915
Epoch 2/20
114/114 [==============================] - 34s 299ms/step - loss: 0.0150 - accuracy: 0.9915
Epoch 3/20
114/114 [==============================] - 34s 298ms/step - loss: 0.0147 - accuracy: 0.9915
Epoch 4/20
114/114 [==============================] - 34s 299ms/step - loss: 0.0145 - accuracy: 0.9916
Epoch 5/20
114/114 [==============================] - 34s 299ms/step - loss: 0.0143 - accuracy: 0.9916
Epoch 6/20
114/114 [==============================] - 34s 299ms/step - loss: 0.0141 - accuracy: 0.9917
Epoch 7/20
114/114 [==============================] - 34s 299ms/step - loss: 0.0139 - accuracy: 0.9917
Epoch 8/20
114/114 [==============================] - 34s 299ms/step - loss: 0.0137 - accuracy: 0.9918
Epoch 9/20
114/114 [==============================] - 34s 298ms/step - loss: 0.0136 - accuracy: 0.9918
Epoch 10/20
114/114 [==============================] - 34s 299ms/step - l

In [ ]:
en_vocab = en_vectorization.get_vocabulary()
en_index_lookup = dict(zip(range(len(en_vocab)), en_vocab))
max_decoded_sentence_length = sequence_length


LSTM_encoder = encoder



decoder_inputs_en_single  = layers.Input(shape=(1,), name="decoder_inputs_en_single")
embeddedLang_en_single =  embeddingLang_en(decoder_inputs_en_single)

decoder_state_input_h = layers.Input(shape=(HIDDEN_DIM,), name="decoder_state_input_h")
decoder_state_input_c = layers.Input(shape=(HIDDEN_DIM,), name="decoder_state_input_c")
decoder_state_input_h1 = layers.Input(shape=(HIDDEN_DIM,), name="decoder_state_input_h1")
decoder_state_input_c1 = layers.Input(shape=(HIDDEN_DIM,), name="decoder_state_input_c1")
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c, 
                         decoder_state_input_h1, decoder_state_input_c1]
layer1, state_h, state_c = decoder_lstm_en(
    embeddedLang_en_single, initial_state=decoder_states_inputs[:2])
layer2, state_h1, state_c1 = decoder_lstm_en_1(
    layer1, initial_state=decoder_states_inputs[-2:])
decoder_states = [state_h, state_c, state_h1, state_c1]
decoder_outputs = decoder_dense_en(layer2)
LSTM_decoder = keras.Model(
    [decoder_inputs_en_single] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

LSTM_decoder.summary()


Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 decoder_inputs_en_single (Inpu  [(None, 1)]         0           []                               
 tLayer)                                                                                          
                                                                                                  
 embedding_5 (Embedding)        multiple             15831040    ['decoder_inputs_en_single[0][0]'
                                                                 ]                                
                                                                                                  
 decoder_state_input_h (InputLa  [(None, 1024)]      0           []                               
 yer)                                                                                       

In [ ]:
embeddingLang_en

In [ ]:

LSTM_decoder.summary()
def decode_sequence(input_sentence):
    tokenized_input_sentence = cs_vectorization([input_sentence])
    vector_set = LSTM_encoder.predict(tokenized_input_sentence)[1:3]
    states_value = vector_set + vector_set
    decoded_sentence = "[start]"
    sampled_token = decoded_sentence
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = en_vectorization([sampled_token])[:,0]
        temp = [tokenized_target_sentence] + states_value
        predictions, h, c, h1, c1 = LSTM_decoder(temp)
        sampled_token_index = np.argmax(predictions[0, -1, :])
        sampled_token = en_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token

        if sampled_token == "[end]":
            break
        states_value = [h, c, h1, c1]
    return decoded_sentence

alltext = []
for i in range(len(train_cs)):
    #input_sentence = random.choice(train_fr)
    translated = decode_sequence(train_cs[i])
    alltext.append(translated)
    print(str(i)+translated)





Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 decoder_inputs_en_single (Inpu  [(None, 1)]         0           []                               
 tLayer)                                                                                          
                                                                                                  
 embedding_5 (Embedding)        multiple             15831040    ['decoder_inputs_en_single[0][0]'
                                                                 ]                                
                                                                                                  
 decoder_state_input_h (InputLa  [(None, 1024)]      0           []                               
 yer)                                                                                       

KeyboardInterrupt: ignored

In [ ]:

LSTM_encoder = fr_encoder

LSTM_decoder.summary()
def decode_sequence(input_sentence):
    tokenized_input_sentence = fr_vectorization([input_sentence])
    vector_set = LSTM_encoder.predict(tokenized_input_sentence)[1:3]
    states_value = vector_set + vector_set
    decoded_sentence = "[start]"
    sampled_token = decoded_sentence
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = en_vectorization([sampled_token])[:,0]
        temp = [tokenized_target_sentence] + states_value
        predictions, h, c, h1, c1 = LSTM_decoder(temp)
        sampled_token_index = np.argmax(predictions[0, -1, :])
        sampled_token = en_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token

        if sampled_token == "[end]":
            break
        states_value = [h, c, h1, c1]
    return decoded_sentence

alltext = []
for i in range(len(train_fr)):
    #input_sentence = random.choice(train_fr)
    translated = decode_sequence(train_fr[i])
    alltext.append(translated)
    print(str(i)+translated)


Streaming output truncated to the last 5000 lines.
24000[start] an older gentleman showing pride as he shows off his gardening harvest [end]
24001[start] a man in black holding a bundle of hay [end]
24002[start] a man on a ladder making something with dried plants [end]
24003[start] two men one in a black shirt the other wearing a jacket piling up wheat [end]
24004[start] a woman in a trench coat hailing a taxi for a ride [end]
24005[start] a brownhaired boy in swim trunks is flipped backwards over a swimming pool [end]
24006[start] a young child splashes in a green and yellow wading pool [end]
24007[start] a man wearing sunglasses and wearing a red vest is standing in front of a red helicopter [end]
24008[start] a girl in a pink dress plays with a hula hoop [end]
24009[start] a kayaker in whitewater rapids between two rocks and a bystander on a rocky shore watching [end]
24010[start] two children water rafting on a lake [end]
24011[start] a little girl is sitting in a metal tub [end]


In [ ]:

similarity_2 = layers.Subtract(name="zero1")([language_enc_1[1], language_enc_2[1]])
similarity_3 = layers.Subtract(name="zero2")([language_enc_1[2], language_enc_2[2]])


decoder_results_5 = decoder_lstm_1(embeddedLang_4, initial_state=language_enc_2[1:3])
decoder_outputs_5 = decoder_dense_1(decoder_results_5[0])
decoder_results_6 = decoder_lstm_2(embeddedLang_3, initial_state=language_enc_2[1:3])
decoder_outputs_6 = decoder_dense_2(decoder_results_6[0])
model_Supervise_de = keras.Model([language_2, language_1, decoder_inputs_1, decoder_inputs_2], [similarity_2, similarity_3, decoder_outputs_5, decoder_outputs_6])
model_Clip_de = keras.Model([language_2, language_1], [similarity_2, similarity_3])
###########################################################################################

batch_size = 128
def format_dataset(cs, de, zero):
    auto_cs = auto_cs_vectorization(cs)
    input_cs = cs_vectorization(cs)
    input_de = de_vectorization(de)
    de = de_de_vectorization(de)
    return ({"encoder_inputs": input_cs, "de_encoder_inputs": input_de, "decoder_inputs": de[:, :-1], "autodecoder_inputs": auto_cs[:, :-1],}, 
            {"de": de[:, 1:], "cs": auto_cs[:, 1:], "zero1": zero, "zero2": zero})


def make_dataset(cs_texts, de_texts):
    cs_texts = list(cs_texts)
    de_texts = list(de_texts)
    zero = np.zeros((len(de_texts), HIDDEN_DIM))
    dataset = tf.data.Dataset.from_tensor_slices((cs_texts, de_texts, zero))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset)
    return dataset.shuffle(2048).prefetch(16).cache()


train_ds_de = make_dataset(train_cs, train_de)

model_Supervise_de.compile(
  loss = [custom_loss, custom_loss, 'sparse_categorical_crossentropy', 'sparse_categorical_crossentropy'],
  optimizer = "adam"
)


model_Supervise_de.summary()

model_Supervise_de.fit(train_ds_de, epochs=0)#, validation_data=train_ds

Model: "model_8"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 de_encoder_inputs (InputLayer)  [(None, 41)]        0           []                               
                                                                                                  
 embedding_1 (Embedding)        (None, 41, 1024)     25507840    ['de_encoder_inputs[0][0]']      
                                                                                                  
 encoder_inputs (InputLayer)    [(None, 35)]         0           []                               
                                                                                                  
 bidirectional_1 (Bidirectional  (None, 41, 1024)    6295552     ['embedding_1[0][0]']            
 )                                                                                          

In [ ]:
epochs = 10  # This should be at least 30 for convergence

model_Supervise_de.summary()
opt = keras.optimizers.Adam(learning_rate=0.0001)
model_Supervise_de.compile(loss = [custom_loss, custom_loss, 'sparse_categorical_crossentropy', 'sparse_categorical_crossentropy'], optimizer=opt)
model_Supervise_de.fit(train_ds_de, epochs=epochs)#, validation_data=train_ds

Model: "model_8"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 de_encoder_inputs (InputLayer)  [(None, 41)]        0           []                               
                                                                                                  
 embedding_1 (Embedding)        (None, 41, 1024)     25507840    ['de_encoder_inputs[0][0]']      
                                                                                                  
 encoder_inputs (InputLayer)    [(None, 35)]         0           []                               
                                                                                                  
 bidirectional_1 (Bidirectional  (None, 41, 1024)    6295552     ['embedding_1[0][0]']            
 )                                                                                          

In [ ]:
epochs = 30

batch_size = 256
def format_dataset(cs, de, zero):
    auto_cs = auto_cs_vectorization(cs)
    input_cs = cs_vectorization(cs)
    input_de = de_vectorization(de)
    de = de_de_vectorization(de)
    return ({"encoder_inputs": input_cs, "de_encoder_inputs": input_de,}, 
            {"zero1": zero, "zero2": zero})


def make_dataset(cs_texts, de_texts):
    cs_texts = list(cs_texts)
    de_texts = list(de_texts)
    zero = np.zeros((len(de_texts), HIDDEN_DIM))
    dataset = tf.data.Dataset.from_tensor_slices((cs_texts, de_texts, zero))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset)
    return dataset.shuffle(2048).prefetch(16).cache()


train_ds_de = make_dataset(train_cs, train_de)


opt = keras.optimizers.Adam(learning_rate=0.0001)

model_Clip_de.compile(
  loss = [custom_loss, custom_loss],
  optimizer=opt
)

model_Clip_de.fit(train_ds_de, epochs=epochs)#, validation_data=train_ds

Epoch 1/30
114/114 [==============================] - 43s 134ms/step - loss: 0.0117 - zero1_loss: 0.0029 - zero2_loss: 0.0089
Epoch 2/30
114/114 [==============================] - 15s 134ms/step - loss: 0.0095 - zero1_loss: 0.0022 - zero2_loss: 0.0073
Epoch 3/30
114/114 [==============================] - 15s 133ms/step - loss: 0.0093 - zero1_loss: 0.0021 - zero2_loss: 0.0072
Epoch 4/30
114/114 [==============================] - 15s 134ms/step - loss: 0.0092 - zero1_loss: 0.0021 - zero2_loss: 0.0071
Epoch 5/30
114/114 [==============================] - 15s 133ms/step - loss: 0.0091 - zero1_loss: 0.0021 - zero2_loss: 0.0071
Epoch 6/30
114/114 [==============================] - 15s 133ms/step - loss: 0.0091 - zero1_loss: 0.0021 - zero2_loss: 0.0070
Epoch 7/30
114/114 [==============================] - 15s 133ms/step - loss: 0.0090 - zero1_loss: 0.0021 - zero2_loss: 0.0069
Epoch 8/30
114/114 [==============================] - 15s 134ms/step - loss: 0.0089 - zero1_loss: 0.0020 - zero2_loss:

In [ ]:
model_Clip_de.load_weights('/content/drive/My Drive/model_Clip_de.h5')

In [ ]:

LSTM_encoder = de_encoder

LSTM_decoder.summary()
def decode_sequence(input_sentence):
    tokenized_input_sentence = de_vectorization([input_sentence])
    vector_set = LSTM_encoder.predict(tokenized_input_sentence)[1:3]
    states_value = vector_set + vector_set
    decoded_sentence = "[start]"
    sampled_token = decoded_sentence
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = en_vectorization([sampled_token])[:,0]
        temp = [tokenized_target_sentence] + states_value
        predictions, h, c, h1, c1 = LSTM_decoder(temp)
        sampled_token_index = np.argmax(predictions[0, -1, :])
        sampled_token = en_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token

        if sampled_token == "[end]":
            break
        states_value = [h, c, h1, c1]
    return decoded_sentence

alltext = []
for i in range(len(val_de)):
    #input_sentence = random.choice(train_fr)
    translated = decode_sequence(val_de[i])
    alltext.append(translated)
    print(str(i)+translated)


Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 decoder_inputs_en_single (Inpu  [(None, 1)]         0           []                               
 tLayer)                                                                                          
                                                                                                  
 embedding_5 (Embedding)        multiple             15831040    ['decoder_inputs_en_single[0][0]'
                                                                 ]                                
                                                                                                  
 decoder_state_input_h (InputLa  [(None, 1024)]      0           []                               
 yer)                                                                                       